In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

In [2]:
feature_names = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
species = ['Setosa', 'Versicolor', 'Virginica']
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

In [3]:
train = pd.read_csv(train_path, names=feature_names, header=0)
test = pd.read_csv(test_path, names=feature_names, header=0)

In [4]:
train_y = train.pop("Species")
test_y = test.pop("Species")
print(train_y)

0      2
1      1
2      2
3      0
4      0
      ..
115    1
116    1
117    0
118    0
119    1
Name: Species, Length: 120, dtype: int64


In [5]:
def input_fn(features, labels, training=True, batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [6]:
feature_columns = []
for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))
print(feature_columns)


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [7]:
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[30,10],
                                        n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\alber\\AppData\\Local\\Temp\\tmpgvbsv9o4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\alber\AppData\Local\Temp\tmpgvbsv9o4\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.3178289, step = 0
INFO:tensorflow:global_step/sec: 291.548
INFO:tensorflow:loss = 1.1367966, step = 100 (0.344 sec)
INFO:tensorflow:global_step/sec: 347.226
INFO:tensorflow:loss = 1.071871, step = 200 (0.290 sec)
INF

In [9]:
eval_result=classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False)
)
print(eval_result["accuracy"])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-10-27T13:44:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\alber\AppData\Local\Temp\tmpgvbsv9o4\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.18700s
INFO:tensorflow:Finished evaluation at 2021-10-27-13:44:13
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.6666667, average_loss = 0.5456292, global_step = 5000, loss = 0.5456292
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\alber\AppData\Local\Temp\tmpgvbsv9o4\model.ckpt-5000
0.6666667


In [16]:
def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

for feature in features:
  val = input(feature + ": ")
  if val.isdigit():
    predict[feature] = [float(val)]

print(predict)

predictions = classifier.predict(input_fn=lambda: input_fn(predict))

for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
                SPECIES[class_id], 100 * probability))


{'SepalLength': [10.0], 'SepalWidth': [12.0], 'PetalLength': [2.0]}
INFO:tensorflow:Calling model_fn.


ValueError: in user code:

    C:\Users\alber\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow_estimator\python\estimator\canned\dnn.py:346 call  *
        net = self._input_layer(features, training=is_training)
    C:\Users\alber\anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\base_layer_v1.py:765 __call__  **
        outputs = call_fn(cast_inputs, *args, **kwargs)
    C:\Users\alber\anaconda3\envs\tf-gpu\lib\site-packages\keras\feature_column\dense_features.py:168 call  **
        tensor = column.get_dense_tensor(transformation_cache,
    C:\Users\alber\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:2697 get_dense_tensor
        return transformation_cache.get(self, state_manager)
    C:\Users\alber\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:2460 get
        transformed = column.transform_feature(self, state_manager)
    C:\Users\alber\anaconda3\envs\tf-gpu\lib\site-packages\tenso